In [1]:
!pip install whisper yt-dlp gradio torchaudio spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
  

In [2]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-su__a5o4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-su__a5o4
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=166d2d505397c1f027aa067e1dc78d5a93a41b4eb55153aaabe3895e7ab9e904
  Stored in directory: /tmp/pip-ephem-wheel-cache-q437ysfj/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper


In [3]:
import os

def download_audio(youtube_url):
    from yt_dlp import YoutubeDL

    output_path = "audio.mp3"

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": "audio.%(ext)s",
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])


    if os.path.exists("audio.mp3.mp3"):
        os.rename("audio.mp3.mp3", "audio.mp3")

    return "audio.mp3"

In [4]:
def transcribe_audio(audio_path, model):
    result = model.transcribe(audio_path, word_timestamps=True)
    return result

In [5]:
import whisper
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:27<00:00, 56.1MiB/s]


In [6]:
import torchaudio
import torchaudio.functional as F

def apply_vad(audio_path):
    wav, sr = torchaudio.load(audio_path)
    vad_audio = F.vad(wav, sample_rate=sr)
    return vad_audio, sr

In [7]:
from pydub.utils import mediainfo

def get_audio_duration(file_path):
    info = mediainfo(file_path)
    return float(info['duration'])  #  seconds

def get_dynamic_chunk_time(total_duration_sec):
    if total_duration_sec < 300:    # less than 5 min
        return 15
    elif total_duration_sec < 900:  # 5–15 min
        return 30
    elif total_duration_sec < 1800: # 15–30 min
        return 45
    else:                           # over 30 min
        return 60

In [8]:
import spacy
import json

def semantic_chunking(transcription, voiced_segments,max_chunk_time):
    nlp = spacy.load("en_core_web_sm")

    segments = []
    current_chunk = ""
    start_time = None
    current_duration = 0

    for segment in transcription["segments"]:
        words = segment["words"]
        text = segment["text"]
        doc = nlp(text)

        # Sentence segmentation
        sentences = [sent.text.strip() for sent in doc.sents]

        for sentence in sentences:
            word_start_time = words[0]["start"]
            word_end_time = words[-1]["end"]

            if start_time is None:
                start_time = word_start_time

            new_duration = word_end_time - start_time


            if new_duration <= max_chunk_time:
                current_chunk += sentence + " "
                current_duration = new_duration
            else:

                if current_chunk.strip():
                    segments.append({
                        "chunk_id": len(segments) + 1,
                        "chunk_length": float(current_duration),
                        "total_time": float(current_duration),
                        "text": current_chunk.strip(),
                        "start_time": float(start_time),
                        "end_time": float(word_end_time)
                    })


                current_chunk = sentence + " "
                start_time = word_start_time
                current_duration = word_end_time - start_time

    # Store the last chunk if it's not empty
    if current_chunk.strip():
        segments.append({
            "chunk_id": len(segments) + 1,
            "chunk_length": float(current_duration),
            "total_time": float(current_duration),
            "text": current_chunk.strip(),
            "start_time": float(start_time),
            "end_time": float(word_end_time)
        })

    return json.dumps(segments, indent=4)


In [9]:
from transformers import pipeline

# Load LLM
llm = pipeline("text2text-generation", model="google/flan-t5-small")
#translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")
translator = pipeline("translation", model="facebook/m2m100_418M", src_lang="en", tgt_lang="hi")
# Define functions for each enhancement
def generate_title(text):
    prompt = f"Generate a short title for this: {text}"
    return llm(prompt, max_length=30)[0]['generated_text']

def summarize(text):
    prompt = f"Summarize in one sentence: {text}"
    return llm(prompt, max_length=50)[0]['generated_text']

def tag_topic(text):
    prompt = f"Identify the topic of this text: {text}"
    return llm(prompt, max_length=10)[0]['generated_text']

def enhanced_chunking(chunked_output):
    enhanced_chunks = []
    for chunk in chunked_output:
        text = chunk['text']

        # LLM enrichment (title, summary, etc.)
        title = generate_title(text)
        summary = summarize(text)
        topic = tag_topic(text)
        translated_text = translator(text, max_length=1024)[0]['translation_text']


        chunk.update({
            "title": title,
            "summary": summary,
            "topic": topic,
            'translated_text_hi':translated_text
        })

        enhanced_chunks.append(chunk)

    return enhanced_chunks


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Device set to use cpu


In [1]:
import gradio as gr
import json

def process_youtube_video(youtube_url):
    try:
        #  1: Download and process audio
        audio_path = download_audio(youtube_url)
        voiced_audio, sr = apply_vad(audio_path)

        #  2: Transcribe audio
        transcription = transcribe_audio(audio_path, model)

        #  3: Get dynamic chunk size
        duration = get_audio_duration(audio_path)
        max_chunk_time = get_dynamic_chunk_time(duration)

        #  4: Semantic chunking
        chunked_output = semantic_chunking(transcription, voiced_audio, max_chunk_time=max_chunk_time)
        chunked_output = json.loads(chunked_output)  # Convert to JSON

        #  5: LLM and translation enrichment
        enhanced_chunks = enhanced_chunking(chunked_output)

        # Handle empty output
        if not enhanced_chunks:
            return "<p style='color:red; font-size:18px;'>⚠️ No transcript chunks found. Try another video.</p>"

        #  Dark gray background for readability
        formatted_chunks = "<div style='font-family:sans-serif; background-color:#1f2937; padding:15px; border-radius:10px;'>"
        for chunk in enhanced_chunks:
            formatted_chunks += f"""
            <div style='border: 2px solid #444; padding: 15px; margin-bottom: 15px; border-radius: 8px;
                        background: #374151; box-shadow: 2px 2px 8px rgba(0,0,0,0.3); color: white;'>
                <h3 style='color:white;'>🔹 {chunk['title']}</h3>
                <p style='color:white; font-size:15px;'><b>⏳ Time:</b> {chunk['start_time']} - {chunk['end_time']} ({chunk['chunk_length']}s)</p>
                <p style='color:white; font-size:15px;'><b>📝 Text:</b> {chunk['text']}</p>
                <p style='color:white; font-size:15px;'><b>📌 Topic:</b> {chunk['topic']}</p>
                <p style='color:white; font-size:15px;'><b>📖 Summary:</b> {chunk['summary']}</p>
                <p style='color:white; font-size:15px;'><b>🌎 Hindi Translation:</b> {chunk['translated_text_hi']}</p>
            </div><br>
            """
        formatted_chunks += "</div>"

        return formatted_chunks

    except Exception as e:
        return f"<p style='color:red;'>⚠️ Error: {str(e)}</p>"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🎬 **YouTube Semantic Chunker**", elem_id="header")
    gr.Markdown("Extract AI-enhanced transcript chunks with topics, summaries, and Hindi translations.", elem_id="subheader")

    with gr.Row():
        youtube_input = gr.Textbox(label="📺 **YouTube Video URL**", placeholder="Paste YouTube link here...")
        run_btn = gr.Button("🔍 **Analyze Video**", elem_id="analyze-btn")

    results = gr.HTML("")

    run_btn.click(fn=process_youtube_video, inputs=[youtube_input], outputs=[results])

demo.launch()


ModuleNotFoundError: No module named 'gradio'